In [1]:
from com.xilinx.rapidwright.design import Design
from com.xilinx.rapidwright.design import DesignTools
from com.xilinx.rapidwright.design import NetType
from com.xilinx.rapidwright.design import Unisim
from com.xilinx.rapidwright.design import Module
from com.xilinx.rapidwright.design import SitePinInst
from com.xilinx.rapidwright.device import Device
from com.xilinx.rapidwright.device import Site
from com.xilinx.rapidwright.device import BEL
from com.xilinx.rapidwright.device import SitePIP
from com.xilinx.rapidwright.device import SiteTypeEnum
from com.xilinx.rapidwright.design import PinType 
from com.xilinx.rapidwright.design import Net 
from com.xilinx.rapidwright.design import NetType
from com.xilinx.rapidwright.edif import EDIFCell
from com.xilinx.rapidwright.edif import EDIFCellInst
from com.xilinx.rapidwright.edif import EDIFDirection
from com.xilinx.rapidwright.edif import EDIFNet
from com.xilinx.rapidwright.edif import EDIFNetlist
from com.xilinx.rapidwright.edif import EDIFPort
from com.xilinx.rapidwright.edif import EDIFPortInst
from com.xilinx.rapidwright.edif import EDIFTools
from com.xilinx.rapidwright.edif import EDIFParser
from com.xilinx.rapidwright.edif import EDIFLibrary
from com.xilinx.rapidwright.edif import EDIFPropertyObject
from com.xilinx.rapidwright.edif import EDIFValueType
from com.xilinx.rapidwright.util import FileTools
from com.xilinx.rapidwright.router import RouteNode
from com.xilinx.rapidwright.router import Router
from com.xilinx.rapidwright.util   import MessageGenerator

### JSONReader.py

In [2]:
import json
import os

def read_file(filename):
    with open(filename, 'r') as file:
        jnet = json.load(file)
    return jnet



def buildLibrary(jnet, library):
    topModule = library.getNetlist().getTopCell().getName()
    for cell in jnet['modules']:
        if cell == topModule: continue
        if library.containsCell(cell): continue
        ret = EDIFCell(library,cell)
        for port in jnet['modules'][cell]['ports'].keys():
            if jnet['modules'][cell]['ports'][port]['direction'] == "input":
                direction = EDIFDirection.INPUT
            elif jnet['modules'][cell]['ports'][port]['direction'] == "output":
                direction = EDIFDirection.OUTPUT
            else:
                direction = EDIFDirection.INOUT
            ret.createPort(port,direction,1)
        library.addCell(ret)


def defineExternalPorts(jnet, topCell):
    topModule = topCell.getName()
    for port in jnet['modules'][topModule]["ports"]:
        port_data = jnet['modules'][topModule]["ports"].get(port)
        port_name = str(port)
        num_pins = len(port_data.get("bits"))
        for i in range(num_pins):
            name = port_name + ("" if num_pins == 1 else ("[" + str(i)  + "]"))
            connectionID = port_data.get("bits")[i]
            direction = port_data.get("direction")
            if direction == "input":
                ret = topCell.createPort(name,EDIFDirection.INPUT, 1)
            elif direction == "output":
                ret = topCell.createPort(name,EDIFDirection.OUTPUT, 1)
            else:
                ret = topCell.createPort(name,EDIFDirection.INOUT, 1)

                

def createEDIFCellInsts(jnet, topCell, library):
    topModule = topCell.getName()
    for cell in jnet['modules'][topModule]["cells"]:
        cell_data = jnet['modules']['top']['cells'].get(cell)
        cell_name = cell
        cell_type = cell_data['type']
        cell_properties = cell_data.get('parameters')
        cell_ports = cell_data['port_directions'].keys()
        cell_port_directions = cell_data['port_directions'].values()
        cell_ports_connectionID = cell_data['connections'].values()
        ret = EDIFCellInst(cell_name, library.getCell(cell_type),topCell)
        for proprty in cell_properties.keys():
            value = cell_properties.get(proprty)
            if value == 'x': continue
            ret.addProperty(proprty,str(int(value,2)), EDIFValueType.INTEGER )


def createEDIFPorts(jnet, topCell):
    topModule = topCell.getName()
    ports = []
    for net in jnet['modules'][topModule]["netnames"]:
        connectionIDs = jnet['modules'][topModule]["netnames"][net]['bits']
        for i in range(len(connectionIDs)):
            ret = topCell.createNet(str(net) + ("" if (len(connectionIDs)) == 1 else ("[" + str(i)  + "]")))
            for cell in jnet['modules'][topModule]["cells"]:
                for port in jnet['modules'][topModule]["cells"][cell]['connections']:
                    value = jnet['modules'][topModule]["cells"][cell]['connections'].get(port)
                    if value[0] == connectionIDs[i]:
                        if ((port,cell) in ports): continue
                        else:
                            ret.createPortInst(port,topCell.getCellInst(cell))
                            ports.append((port,cell))
            for port in jnet['modules'][topModule]['ports']:
                topConnectionIDs = jnet['modules'][topModule]['ports'][port]['bits']
                for j in range(len(topConnectionIDs)):
                    if topConnectionIDs[j] == connectionIDs[i]:
                        name = str(port) + ("" if (len(topConnectionIDs)) == 1 else ("[" + str(j)  + "]"))
                        if ((name,topModule) in ports): continue
                        else:
                            ret.createPortInst(topCell.getPort(name))
                            ports.append((name,topModule))

                            
def cleanEmptyNets(topCell):
    nets = []
    for net in topCell.getNets():
        if not (net.getPortInsts()):
            nets.append(str(net))
    for i in nets:
        topCell.removeNet(i)

        

def createStaticSourceNets(jnet, topCell, netlist):
    topModule = topCell.getName()
    gnd = EDIFTools.getStaticNet(NetType.GND, topCell, netlist);
    vcc = EDIFTools.getStaticNet(NetType.VCC, topCell, netlist);
    for cell in jnet['modules'][topModule]["cells"]:
        for port in jnet['modules'][topModule]["cells"][cell]['connections']:
            value = jnet['modules'][topModule]["cells"][cell]['connections'].get(port)
            if value[0] == "0":
                gnd.createPortInst(port , topCell.getCellInst(cell))
            elif value[0] == "1":
                vcc.createPortInst(port , topCell.getCellInst(cell))


def read_json (filename, topModule, device):
    jnet = read_file(filename)
    design = Design(topModule,device)
    netlist = design.getNetlist()
    topCell = netlist.getTopCell()
    library = netlist.getLibrary("hdi_primitives")
    buildLibrary(jnet, library)
    defineExternalPorts(jnet, topCell)    
    createEDIFCellInsts(jnet, topCell, library)
    createEDIFPorts(jnet, topCell)
    cleanEmptyNets(topCell)
    createStaticSourceNets(jnet, topCell,netlist)
    return design



### ConstrainDesign.py

In [3]:
def placeIOBuffers(design , constraints):
    for pin in constraints.keys():
        for topnets in design.getNetlist().getTopCell().getNets().toArray().tolist():
            if pin == str(topnets):
                for cellName in design.getNetlist().getTopCell().getCellInsts().toArray().tolist():
                    if ("IBUF" in str(cellName.getCellType())) or ("OBUF" in str(cellName.getCellType())):
                        for entry in topnets.getPortInstMap():
                            if str(cellName) in str(entry):
                                ret = design.placeIOB(cellName, constraints.get(pin).get('LOC'), constraints.get(pin).get('IOSTANDARD'))
                                ret.addProperty("IOSTANDARD",constraints.get(pin).get("IOSTANDARD"))
                                ret.addProperty("SLEW",constraints.get(pin).get("SLEW"))
                                ret.addProperty("IN_TERM",constraints.get(pin).get("IN_TERM"))
                                ret.addProperty("PULL_TYPE",constraints.get(pin).get("PULL_TYPE"))

### Logical2PysicalCells.py

In [4]:
def placeCells(design):
    cell_Names = design.getNetlist().getTopCell().getCellInsts().toArray().tolist()
    placement_list = []
    for cn in cell_Names:
        if ("GND" in str(cn.getCellType())) or ("VCC" in str(cn.getCellType())):
             continue

        elif ("IBUF" in str(cn.getCellType())) or ("OBUF" in str(cn.getCellType())):
            continue

        elif ("BUFGCTRL" in str(cn.getCellType())):
            design.getNetlist().getTopCell().removeCellInst(str(cn))
            loc = "BUFGCTRL_X0Y16/BUFGCTRL"
            ret = design.createAndPlaceCell(str(cn),Unisim.valueOf(str(cn.getCellType())),loc) 
            ret.setProperties(cn.getProperties())
            ret.connectStaticSourceToPin(NetType.VCC ,"CE0")
            ret.connectStaticSourceToPin(NetType.VCC ,"S0")
            ret.connectStaticSourceToPin(NetType.GND ,"CE1")
            ret.connectStaticSourceToPin(NetType.GND ,"S1")
            ret.connectStaticSourceToPin(NetType.GND ,"IGNORE1")
            ret.connectStaticSourceToPin(NetType.GND ,"IGNORE0")


        elif ("FDRE" in str(cn.getCellType())): 
            bels = Design().createCell(str(cn),cn).getCompatiblePlacements().get(SiteTypeEnum.SLICEL)
            sites = design.getDevice().getAllCompatibleSites(SiteTypeEnum.SLICEL)    
            running = 1
            while running:
                bel_name = list(bels)[random.randint(0,len(bels)-1)]
                site = sites[random.randint(0,len(sites)-1)]
                bel = site.getBEL(bel_name)
                if (str(site)+"/"+str(bel)) not in placement_list:
                    placement_list.append((str(site)+"/"+str(bel))) 
                    running = 0
            loc = str(site)+"/"+str(bel.getName())
            unsm = Unisim.FDRE
            cell = str(cn)
            design.getNetlist().getTopCell().removeCellInst(str(cn))
            ret = design.createAndPlaceCell(cell,unsm,loc)        
            ret.connectStaticSourceToPin(NetType.GND ,"R")
            ret.connectStaticSourceToPin(NetType.VCC ,"CE")

        elif ("LUT" in str(cn.getCellType())):
            cell = design.createCell(str(cn),cn)
            bels = cell.getCompatiblePlacements().get(SiteTypeEnum.SLICEL)
            sites = design.getDevice().getAllCompatibleSites(SiteTypeEnum.SLICEL)    
            running = 1
            while running:
                bel_name = list(bels)[random.randint(0,len(bels)-1)]
                site = sites[random.randint(0,len(sites)-1)]
                bel = site.getBEL(bel_name)
                if (str(site)+"/"+str(bel)) not in placement_list:
                    placement_list.append((str(site)+"/"+str(bel))) 
                    running = 0
                    design.placeCell(cell,site,bel)
print ""

### Logical2PhysicalNets.py

In [5]:
def createNets(design):
    for i in design.getNets().toArray().tolist():
        design.removeNet(i)
    for i in design.getNetlist().getTopCell().getNets().toArray().tolist():
        if str(i) in design.getNetlist().getTopCell().getPortMap().keys() and not(str(i) == "clk"): continue  
        if str(i) == "VCC_NET": continue     
        if str(i) == "GND_NET": continue   
        design.createNet(i)
    
    
def createNetPins(design):
    for physNet in design.getNets().toArray().tolist():
        if str(physNet) in design.getNetlist().getTopCell().getPortMap().keys() and not(str(physNet) == "clk"): continue
        edifNet = physNet.getLogicalNet()
        portInsts = edifNet.getPortInsts().toArray().tolist()
        for portInst in portInsts:
            portName = portInst.getName()
            portCell = portInst.getCellInst()
            portDir = portInst.getDirection()
            physCell = design.getCell(str(portCell))
            if portInst.isPrimitiveStaticSource(): continue
            siteInst = physCell.getSiteInst()
            siteWires = []
            ret = physCell.getSitePinFromPortInst(portInst,siteWires)            
            physNet.createPin(portInst.isOutput(),siteWires[len(siteWires)-1],siteInst)

### SiteRouter.py

In [6]:
def routeSitePIPs(design):
    for edifNet in design.getNetlist().getTopCell().getNets().toArray().tolist():
        if str(edifNet) in design.getNetlist().getTopCell().getPortMap().keys() and not(str(edifNet) == "clk"): continue
        if  (str(edifNet) == "GND_NET"): continue
        portInsts = edifNet.getPortInsts().toArray().tolist()
        for portInst in portInsts:
            if portInst.isPrimitiveStaticSource(): continue
            portCell = portInst.getCellInst()
            physCell = design.getCell(str(portCell))
            siteInst = physCell.getSiteInst()
            if "BUFGCTRL" in str(physCell.getType()):
                if (not("CE0" in str(portInst.getName()))) and (not("S0" in str(portInst.getName()))): continue
                siteWires = []
                ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                bel = siteWires[0].split("_")[0] 
                belpin = ""
                for bp in siteInst.getSite().getBELPins(siteWires[1]).tolist():
                    if str(bel) in str(bp):
                        belpin = bp
                        break
                sitepip = siteInst.getSitePIP(belpin)
                siteInst.addSitePIP(sitepip)
                siteInst.routeSite()
            elif "FDRE" in str(physCell.getType()):
                if ("CE" == str(portInst.getName())):
                    belpin = siteInst.getSite().getBEL("CEUSEDMUX").getPin("1")
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()  
                elif ("R" == str(portInst.getName())):
                    belpin = siteInst.getSite().getBEL("SRUSEDMUX").getPin("0")
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()  
                elif ("C" == str(portInst.getName())):
                    belpin = siteInst.getSite().getBEL("CLKINV").getPin("CLK")
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()
                elif  ("Q" == str(portInst.getName())):
                    siteWires = []
                    ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                    if len(siteWires)>1:
                        belpin = ""
                        for bp in siteInst.getSite().getBELPins(siteWires[0]).tolist():
                            if str(siteInst.getSite().getBELPins(siteWires[1]).tolist()[0]).split(".")[0] in str(bp):
                                belpin = bp
                                break
                        sitepip = siteInst.getSitePIP(belpin)
                        siteInst.addSitePIP(sitepip)
                        siteInst.routeSite()                
                elif  ("D" == str(portInst.getName())):
                    siteWires = []
                    ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                    if len(siteWires)>1:
                        bel = siteWires[0].split("_")[0] 
                        belpin = ""
                        for bp in siteInst.getSite().getBELPins(siteWires[1]).tolist():
                            if str(bel) in str(bp):
                                belpin = bp
                                break
                        sitepip = siteInst.getSitePIP(belpin)
                        siteInst.addSitePIP(sitepip)
                        siteInst.routeSite()
            elif "LUT" in str(physCell.getType()):
                siteWires = []
                ret = physCell.getSitePinFromPortInst(portInst,siteWires)
                if len(siteWires)>1 and portInst.isOutput():
                    if ("IOB" in str(siteInst.getSite().getSiteTypeEnum())): continue
                    belpin = ""
                    for bp in siteInst.getSite().getBELPins(siteWires[0]).tolist():
                        if str(siteInst.getSite().getBELPins(siteWires[1]).tolist()[0]).split(".")[0] in str(bp):
                            belpin = bp
                            break
                    sitepip = siteInst.getSitePIP(belpin)
                    siteInst.addSitePIP(sitepip)
                    siteInst.routeSite()
print ""

### DesignClockRouter.py

In [7]:
from java.util import HashSet
from java.util import List
from pprint import pprint
import Queue
from Queue import PriorityQueue

def cost (s1, s2):
    return abs(s1.getRpmX() - s2.getRpmX()) + abs(s1.getRpmY() - s2.getRpmY())


def closestSite (s1, possible_sites):
    p = PriorityQueue()
    for s2 in possible_sites:
        p.put((cost(s1,s2),s2))
    return p.get()

def createBUFHs(design):
    if (design.getNet("clk")):
        sites = []
        pins = design.getNet("clk").getSinkPins()
        for pin in pins:
            pin_site = pin.getSite()
            bufhce_sites =  design.getDevice().getAllCompatibleSites(SiteTypeEnum.BUFHCE).tolist()
            sites.append(closestSite(pin_site,bufhce_sites)[1])

        for i in range(len(pins)):
            exists = -1
            for j in range(i):
                if design.getCell("bufhce_"+str(j)):
                    if str(design.getCell("bufhce_"+str(j)).getSite()) == str(sites[i]):
                        exists = j

            if exists == -1:
                loc = str(sites[i]) +"/BUFHCE"
                ret = design.createAndPlaceCell("bufhce_"+str(i),Unisim.BUFHCE,loc) 
                ret.connectStaticSourceToPin(NetType.VCC ,"CE")
                ret.getSiteInst().addSitePIP(ret.getSiteInst().getSitePIP(ret.getSiteInst().getBEL("CEINV").getPin("CE")))
                ret.getSiteInst().routeSite()
                net = design.createNet("clk" + str(i))
                design.getNet("clk").removePin(pins[i])
                design.getNet("clk").addPin(SitePinInst("I",ret.getSiteInst()))
                net.addPin(SitePinInst("O",ret.getSiteInst()))
                net.addPin(pins[i])
                topCell.getNet("clk"+str(i)).createPortInst("O",ret)
                topCell.getNet("clk"+str(i)).addPortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])
                topCell.getNet("clk").createPortInst("I",ret)
                topCell.getNet("clk").removePortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])
            else:
                net = design.getNet("clk" + str(exists))
                design.getNet("clk").removePin(pins[i])
                net.addPin(pins[i])
                topCell.getNet("clk"+str(exists)).addPortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])
                topCell.getNet("clk").removePortInst(topCell.getNet("clk").getPortInsts().toArray().tolist()[0])


def costFunction(curr, snk):
    return curr.getManhattanDistance(snk) + curr.getLevel()/8  

def routeClockNet(net, usedPIPs):
    path = []
    for sink in net.getPins():
        if sink.equals(net.getSource()): 
            continue
        q = RouteNode.getPriorityQueue()
        q.add(net.getSource().getRouteNode())
        path.extend(findRoute(q,sink.getRouteNode())) 
    path = list(dict.fromkeys(path))
    net.setPIPs(path)
    return


def findRoute(q, snk):
    visited = HashSet()
    src = q.poll()
    base_cost = costFunction(src,snk)
    q.add(src)
    counter = 0
    while(not q.isEmpty()):
        curr = q.poll()
        if(curr.equals(snk)):
            return curr.getPIPsBackToSource()
        visited.add(curr)


        for wire in curr.getConnections():
            nextNode = RouteNode(wire,curr)
            if visited.contains(nextNode): continue
            #if wire.isRouteThru(): continue
            curr_cost = costFunction(nextNode,snk)
            nextNode.setCost(curr_cost)
            q.add(nextNode)
    # Unroutable situation
    print "Route failed!"
    return []


def routeClocks(design):
    usedPIPs = []
    createBUFHs(design)
    for net in design.getNets():
        if "clk" in str(net):  
            routeClockNet(net,usedPIPs)

### DesignRouter.py

In [8]:
def designRouter(design):
    for net in design.getNets():
            if "clk" in str(net):  
                net.lockRouting()

    Router(design).routeDesign()

    for net in design.getNets():
        if "clk" in str(net):
            net.unlockRouting()

### run.py

In [9]:
import random
import os

def run(seed, topModule,device,filename,constraints):
    random.seed(seed)
    design = read_json(filename,topModule,device)
    net = design.getNetlist()
    net.setDevice(design.getDevice())
    topCell = net.getCell("top")

    placeIOBuffers(design , constraints)
    placeCells(design)
    createNets(design)
    createNetPins(design)
    routeSitePIPs(design)
    design.routeSites() # not necessary, but helps route input site wire for OBUFS
    routeClocks(design)
    try:
        designRouter(design)
        outputFileName = topModule+"6.dcp"
        design.writeCheckpoint(outputFileName)
        print "Wrote DCP '" + os.path.join(os.getcwd(), outputFileName) + "' successfully"
        return 1
    except:
        return 0

### Utilities.py

In [24]:
import os

def run_yosys(topModule):
    yosys_cmd = """yosys -p "synth_xilinx -flatten -abc9 -nobram -arch xc7 -top """+topModule+"""; write_json """+topModule+""".json" """+topModule+""".v"""
    ret = os.system(yosys_cmd)
    

def generate_bistream(topModule):
    tcl_code="""
    open_checkpoint """+topModule+""".dcp
    write_bitstream -force """+topModule+"""
    """
    tcl_file = open(topModule+".tcl","w")
    ret = tcl_file.write(tcl_code)
    tcl_file.close()
    vivado_cmd = """vivado vivado_files/"""+topModule+""".xpr -nolog -nojournal -notrace -mode batch -source """+topModule+""".tcl"""
    ret = os.system(vivado_cmd)

### Main.py

In [26]:
topModule = "top"
device = "xc7a35tcsg324-1"
filename = "top.json"

constraints = {
               'i_clk' : {'LOC' : 'E3', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in0[1]' : {'LOC' : 'C11', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in0[0]' : {'LOC' : 'A8', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in1[1]' : {'LOC' : 'A10', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'in1[0]' : {'LOC' : 'C10', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[3]' : {'LOC' : 'K1', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[2]' : {'LOC' : 'J3', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[1]' : {'LOC' : 'G3', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'},
               'out[0]' : {'LOC' : 'G6', 'IOSTANDARD' : 'LVCMOS33' , 'SLEW' : 'SLOW', 'IN_TERM' : 'NONE' , 'PULL_TYPE' : 'NONE'}
                }

run_yosys(topModule)

for i in range(1000):
    if (run(i, topModule,device,filename,constraints)):
        print "Seed Value: " + str(i)
        break

generate_bistream(topModule)

Wrote DCP '/home/asanaull/Desktop/Software/RAPIDWRIGHT/demos/yosys/top.dcp' successfully
Seed Value: 2
